# Linking named entities
Filip Gregora

In [208]:
import pandas as pd

data = pd.read_csv("data/NER_entities.csv")

In [209]:
data.head()

,label,text
0,symptom,jemný fibrózní proužek
1,procedura,neoadjuvantní CHT
2,medikace,Novalgin
3,symptom,Označena SLU v levé axile.
4,procedura,st.p. totální ME + SNB vlevo


In [215]:
symptom = data[data.label == "symptom"].drop(["label"], axis=1)
procedura = data[data.label == "procedura"].drop(["label"], axis=1)
medikace = data[data.label == "medikace"].drop(["label"], axis=1)
ne_symptom = data[data.label == "NE symptom"].drop(["label"], axis=1)
os_anamneza = data[data.label == "osobní anamnéza"].drop(["label"], axis=1)
ne_os_anamneza = data[data.label == "NE osobní anamnéza"].drop(["label"], axis=1)
ne_medikace = data[data.label == "NE medikace"].drop(["label"], axis=1)

In [51]:
assert(len(os_anamneza) + len(symptom) + len(procedura) + len(medikace) + len(ne_symptom) + len(ne_os_anamneza) + len(ne_medikace) \
            == len(data))

In [66]:
symptom.head(10)

0          jemný fibrózní proužek
3      Označena SLU v levé axile.
18         mamma l.sin. dysplazie
24                další ojedinělé
29         stále brnění HKK i DKK
36           hmatná tu léze v DKK
59    další velikostní regrese TU
60                      nespavost
64         vlevo jizva ZHK změklá
85    bolesti patní ostruhy vlevo
Name: text, dtype: object

In [93]:
def clean_db(db):
    return db.apply(lambda x: x.strip(" ,.").lower())

In [105]:
symptom = clean_db(symptom)
symptom.head(10)

0          jemný fibrózní proužek
3       označena slu v levé axile
18         mamma l.sin. dysplazie
24                další ojedinělé
29         stále brnění hkk i dkk
36           hmatná tu léze v dkk
59    další velikostní regrese tu
60                      nespavost
64         vlevo jizva zhk změklá
85    bolesti patní ostruhy vlevo
Name: text, dtype: object

In [247]:
import requests
from bs4 import BeautifulSoup
import json
from itertools import combinations

def mash_search(string):
    splitted_input = (string.split(" "))
    result = []
    for j in range(len(splitted_input), 0, -1):
        for string in combinations(splitted_input, j):      
            path = 'https://uts-ws.nlm.nih.gov/rest/search/current'
            query = {
                     'string': " ".join(string),
                     'apiKey':'6a290909-c0d8-4db9-b531-7387929b334e',
            }
            res = requests.get(path, params=query)

            if res.status_code <= 200:
                data = json.loads(res.text)
                for j in data["result"]["results"]:
                    result.append((j["ui"], j["name"]))
            else:
                print(res.status_code, res.text)
        
        if len(result) != 0:
            break
            
    return result
        
r = mash_search('bolesti patní ostruhy vlevo')
print(r)
# print(json.dumps(r, indent=4))

[('C0149756', 'Fasciitis, Plantar')]


In [ ]:
symptom["search"] = symptom["text"].apply(mash_search)

In [ ]:
symptom

In [54]:
import xml.etree.ElementTree as elt

content = elt.parse('databaze/MeSH2023_Marc21.xml').getroot()

In [104]:
for child in content:
#     print(child)
    pass
    